In [106]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import IPython.display as ipd

In [107]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [108]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'neutral',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

In [109]:
# Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("data\\ravdessdata\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
#         if emotion not in observed_emotions:
#             continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [110]:
# Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.20)

In [111]:
# Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(1152, 288)


In [112]:
# Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [147]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500, activation = 'relu')

In [148]:
# Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [149]:
model.score(x_test, y_test)

0.5833333333333334

In [96]:
# Predict for the test set
y_pred=model.predict(x_test)
y_pred

array(['disgust', 'fearful', 'surprised', 'neutral', 'surprised',
       'fearful', 'surprised', 'fearful', 'surprised', 'angry', 'neutral',
       'disgust', 'fearful', 'neutral', 'neutral', 'neutral', 'neutral',
       'fearful', 'sad', 'surprised', 'surprised', 'surprised', 'fearful',
       'fearful', 'fearful', 'surprised', 'neutral', 'neutral', 'neutral',
       'surprised', 'neutral', 'neutral', 'fearful', 'fearful', 'happy',
       'neutral', 'neutral', 'surprised', 'neutral', 'fearful',
       'surprised', 'neutral', 'happy', 'neutral', 'angry', 'neutral',
       'neutral', 'disgust', 'fearful', 'neutral', 'surprised', 'angry',
       'fearful', 'fearful', 'fearful', 'neutral', 'angry', 'disgust',
       'surprised', 'surprised', 'surprised', 'sad', 'angry', 'neutral',
       'fearful', 'surprised', 'sad', 'neutral', 'fearful', 'neutral',
       'fearful', 'angry', 'angry', 'surprised', 'fearful', 'neutral',
       'neutral', 'fearful', 'neutral', 'surprised', 'surprised',
   

In [97]:
pd.DataFrame({'Prediction': y_pred, 'Actual': y_test}).head(60)

,Prediction,Actual
0,disgust,fearful
1,fearful,sad
2,surprised,happy
3,neutral,neutral
4,surprised,happy
5,fearful,angry
6,surprised,surprised
7,fearful,sad
8,surprised,happy
9,angry,angry


In [98]:
# model.predict(x_test[:1])

In [99]:
# x_test[0]

In [100]:
# Play audio file within my script
ipd.Audio('data/kg.wav')

In [101]:
kg, sr = librosa.load('data/kg.wav')

In [102]:
kg

array([ 0.000000e+00,  0.000000e+00,  0.000000e+00, ..., -5.900278e-05,
       -6.436605e-05,  0.000000e+00], dtype=float32)

In [103]:
#----------------------------#
#    Personal Audio test     #
#----------------------------#
feature22=extract_feature('data/kg.wav', mfcc=True, chroma=True, mel=True)
# feature22
# ValueError: Expected 2D array, got 1D array instead:
# array=[-326.22702026].
# Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
kg_predit = model.predict(feature22.reshape(1, -1))
kg_predit

array(['angry'], dtype='<U9')

In [104]:
kg_predit[0]

'angry'

In [105]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 40.28%


In [45]:
# feature2=extract_feature('data/ravdessdata/Actor_16/03-01-01-01-01-01-16.wav', mfcc=True, chroma=True, mel=True)
# feature2